
# About the recommendation engine

We decided to make a network which simulated a search engine. the idea is that a user inputs a name of a movie (node) and from that movie (node) we find all the shortest distances in our network and recommend those movies (nodes) wich are closest in the network. It was decided to use number of things to build this network, in the persude to make as good recommendation engine as possible. 

We decided to impliment a weighted network, since nodes which are connected can have diffrent improtance. For example, if we have some movie, lets say Iron Man, our network can  have multiple connections to that movie. However, these movies are not all equally similar to Iron Man, and thus we add a weight to the edge. Lastly a movie with few edges can be similar to some movies which are not connected to them sel. But there might be a short path in the network which connectes those movies. 

    

## Import Libraries



In [1]:
from __future__  import division
import csv
import math
import urllib2
import json
import re
import os
import operator
import calendar
import json
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import random
import time
import string

from scipy import spatial
from datetime import datetime
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer

## Get the datasets

In [51]:
# Get data line by line from tmdb-5000-movie-database and save the data as dictionary
filepath = r"C:\Users\helga\Dropbox\Ingvar\FinalProject\Data\tmdb-5000-movie-dataset\tmdb_5000_credits.csv"
tmdb_5000_credits = defaultdict(dict)

with open(filepath) as csvfile:
    reader = csv.DictReader(csvfile, delimiter=",")
    for row in reader:
        # we take out punctuation and put the data to lower case in order to make it easier to work with the resulting json in
        # the webpage, then we can make the search case and punctuation in-sensitive
        tmdb_5000_credits[row["title"].lower().translate(None, string.punctuation)] = row
        # put the lists and dictonaries to a correct format, (that is not as string)
        tmdb_5000_credits[row["title"].lower().translate(None, string.punctuation)]['cast'] = json.loads(tmdb_5000_credits[row["title"].lower().translate(None, string.punctuation)]['cast'])
        tmdb_5000_credits[row["title"].lower().translate(None, string.punctuation)]['crew'] = json.loads(tmdb_5000_credits[row["title"].lower().translate(None, string.punctuation)]['crew'])
    


for title in sorted(tmdb_5000_credits.keys()):
    title = title.replace(" ","-").replace("(","").replace(")","")

In [52]:
# Get data line by line from tmdb-5000-movie-database and save the data as dictionary

filepath = r"C:\Users\helga\Dropbox\Ingvar\FinalProject\Data\tmdb-5000-movie-dataset\tmdb_5000_movies.csv"
tmdb_5000_movies = defaultdict(dict)

with open(filepath) as csvfile:
    reader = csv.DictReader(csvfile, delimiter=",")
    for row in reader:
        tmdb_5000_movies[row["title"].lower().translate(None, string.punctuation)] = row
        # put the lists and dictonaries to a correct format, (that is not as string)
        tmdb_5000_movies[row["title"].lower().translate(None, string.punctuation)]['genres'] = json.loads(tmdb_5000_movies[row["title"].lower().translate(None, string.punctuation)]['genres'])
        tmdb_5000_movies[row["title"].lower().translate(None, string.punctuation)]['keywords'] = json.loads(tmdb_5000_movies[row["title"].lower().translate(None, string.punctuation)]['keywords'])
        tmdb_5000_movies[row["title"].lower().translate(None, string.punctuation)]['production_companies'] = json.loads(tmdb_5000_movies[row["title"].lower().translate(None, string.punctuation)]['production_companies'])
        
for title in sorted(tmdb_5000_movies.keys()):
    title = title.replace(" ","-").replace("(","").replace(")","")
    


In [4]:
tmdb_5000_movies.keys()

['our family wedding',
 'the secret',
 'goldfinger',
 'grudge match',
 'aliens in the attic',
 'igby goes down',
 'bobby jones stroke of genius',
 'semipro',
 'girls gone dead',
 'friday the 13th the final chapter',
 'jeepers creepers',
 'oceans',
 'the revenant',
 'doctor dolittle',
 'a history of violence',
 '10 days in a madhouse',
 'into the grizzly maze',
 'the corruptor',
 'dangerous liaisons',
 'the blade of don juan',
 'raising victor vargas',
 'blade: trinity',
 'pulse',
 'valiant',
 'the texas chainsaw massacre: the beginning',
 'hansel & gretel: witch hunters',
 'hidden away',
 'i frankenstein',
 'paranormal activity: the marked ones',
 'hero',
 "don't say a word",
 'tales from the hood',
 'the mudge boy',
 'under siege 2 dark territory',
 'kids',
 'star trek v: the final frontier',
 'cats & dogs 2 : the revenge of kitty galore',
 'the to do list',
 'wreck-it ralph',
 'america is still the place',
 'i am wrath',
 'crywolf',
 "ripley's game",
 'crazy, stupid, love.',
 'housef

## Making the Network

The main network is build upon our analysis on simpler networks. After investigatin both  the keyword network and the movie actor network, it was clear that those networks where not sufficent. Thus a more complex network was build, using more dimensions to make edges in the network. 

In order for making a link in the network, some similarities between the movies (nodes) must exist. It was decided that what mattered in order to make a link between the networks was the number of common: keywords, genre, actors and directors. In addtion to that the distance between the nodes where deceded with the number of common: keywords, genre, actors, directors, IMDB raiting and how similar the storyline is.

### Calculating similarities between storylines

previously (see NOTBOOOKKK ) scraped all the storylines from imdb. Using this information we can caluclate the similarities between the storylines. To do this we use tf-idf and later one use the BLA distance measure to calculate the similarities.  The tf-idf gives us a vector for each storyline which tells us how important each word is. This results in a matrix for all the storylines, we then take each vector and calculate the similarity between the storylines later on in the notebook. Note, to be able to acess the correct vectors at a given time we introduce the dictonary which maps each movie with the correct line of the matrix. 

In [53]:
filepath_storyline = r"C:\Users\helga\Dropbox\Ingvar\Results\imdb-storyline2.json"
#filepath_storyline = r"C:\Users\helga\OneDrive\Documents\social_final\imdb-storyline.json"

# initalize the data
files = []
movie_number = {}

# found from blGRRRRRRRRRRRRRREEEEEEEEEEEEEEETTTTTTTTTTTTAAAAAAAAAAARRRRRRRRRRRRRRRR
# and the last three do not have imdb rating
movies_to_delete =[
    'National Lampoon’s Van Wilder'.translate(None, string.punctuation),
    'Bran Nue Dae'.translate(None, string.punctuation),
    'Pokémon: Spell of the Unknown'.translate(None, string.punctuation),
    'Alien³'.translate(None, string.punctuation),
    'Guten Tag, Ramón'.translate(None, string.punctuation),
    'xxx: state of the union'.translate(None, string.punctuation),
    'xxx'.translate(None, string.punctuation),
    "Æon Flux".lower()
    
]

# load the data
with open(filepath_storyline) as json_data:
    d = json.load(json_data)



index = 0
for movie in tmdb_5000_movies.keys():
    
    _id = tmdb_5000_movies[movie]['id']
    if d[_id] != 0 and d[_id] != None:
        files.append(d[_id])
        movie_number[movie] = index
        index += 1
    else:
        # if a mmovie dose not have a storyline we throw it
        movies_to_delete.append(movie)
        
    



# Note that we excluded the stop words from the data because we did not consider it to be relevant
vectorizer = TfidfVectorizer(input = 'content', analyzer='word', min_df = 0, stop_words = 'english')
TF_IDF_matrix = vectorizer.fit_transform(files).todense()

print "Similarity matrix ready"

Similarity matrix ready


### Building the Network

When making the network various decision regarding what constraints needed to be fullfill  to make a edge and what the weight of that edge should be occured. We start the discussin on the constraint. The constraint build´s upon four dimensions: genere, keywords, actors and director. All of these things are calculated in a similar fashion. We find all the comon things in both movies, be it genere, keyword, actor or director and then normalize it by dividing by the movie which has less items in each dimension. Each of these values are then multiplied by importance coefficent, and if the total sum of the dimension varibles times the importance coefficent is more than 0.3 we make a edge. The value of the importance was choosen by feeling. Since there is no correct answer to which movies you recommend. However, there was still some thorught put to it. Firstly, note that all the dimension varibles can at maximum have value of 1. Next when looking at the importance constraints, one can observe that non of them has a value greater than 0.3. This foreces our network to make a connection only if at least two dimension variables are in use. Next when assigning a weight to the edge we subtract both the sum of the varibles which sastisfied the constraint and the similarities between the movies with one. This way, movies that have similar keywords, genere, actors, directors and storylines are closer in the network. Note that we use the similarity between storylines only in deciding on the weight of the edge.
 The reason for this is that altought the movies might have similar storyline the movie could be very different. For example a romance and a horrors story might mix, depending in the language which is used on IMDB. Furthermore, there exists only so few  [storylines]( https://en.wikipedia.org/wiki/The_Seven_Basic_Plots).
 


In [54]:
# lets build the second network
G = nx.Graph()

# all movies
movies = [movie for movie in tmdb_5000_movies.keys() if movie not in  movies_to_delete]

# add the nodes
G.add_nodes_from(movies)

# ratio of similar keywords that movies have to pass to be consider similar
min_ratio = 0.3
total_ratios = []
links = []
# loop thorugh all the pairs and check if we have enough evidance to insert a link
for i in xrange(len(movies) - 1):
    
    # extract all the keywords from the first movie
    keywords_i = [keyword_i['id'] for keyword_i in tmdb_5000_movies[movies[i]]['keywords']]
    
    # extract the genere from the first movie
    genre_i = [genre_i['id'] for genre_i in tmdb_5000_movies[movies[i]]['genres']]

    # extract all the actors from the first movie
    actors_i = [actor_i['id'] for actor_i in tmdb_5000_credits[movies[i]]['cast']]
    
    # extract all the directors from the first movie
    director_i = [member_i['id'] for member_i in tmdb_5000_credits[movies[i]]['crew'] if member_i['job'] == 'Director']
    
    for j in xrange(i + 1, len(movies)):

        # all the keywords from the second movie
        keywords_j = [keyword_j['id'] for keyword_j in tmdb_5000_movies[movies[j]]['keywords']]
        
        # extract the genere from the second movie
        genre_j = [genre_j['id'] for genre_j in tmdb_5000_movies[movies[j]]['genres']]

        # extract all the actors from the second movie
        actors_j = [actor_j['id'] for actor_j in tmdb_5000_credits[movies[j]]['cast']]

        # extract all the directors from the second movie
        director_j = [member_j['id'] for member_j in tmdb_5000_credits[movies[j]]['crew'] if member_j['job'] == 'Director']
        
        
        
        # ensure that we dont divide by zero
        min_keyword_length = min(len(keywords_i), len(keywords_j))
        if min_keyword_length != 0:
            
            # claculate the keyword ratio of the two movies, using the smaller keywords list
            keyword_ratio = len(set(keywords_i).intersection(set(keywords_j))) / min_keyword_length
        else:
            keyword_ratio = 0.0
            
            
        
                
        #-----------------------------------------------------------------    
                
         # ensure that we dont divide by zero
        min_genre_length = min(len(genre_i), len(genre_j))
        if min_genre_length != 0:
            
            # claculate the keyword ratio of the two movies, using the smaller keywords list
            genre_ratio = len(set(genre_i).intersection(set(genre_j))) / min_genre_length
        else:
            genre_ratio = 0.0
            
        
        #------------------------------------------------------------------------------
        
        # ensure that we dont divide by zero
        min_actors_length = min(len(actors_i), len(actors_j))
        if min_actors_length != 0:
            
            # claculate the keyword ratio of the two movies, using the smaller keywords list
            actors_ratio = len(set(actors_i).intersection(set(actors_j))) / min_actors_length
        else:
            actors_ratio = 0.0
        
            
        #------------------------------------------------------------------------------    
            
        # ensure that we dont divide by zero
        min_director_length = min(len(director_i), len(director_j))
        if min_director_length != 0:
            
            # claculate the keyword ratio of the two movies, using the smaller keywords list
            director_ratio = len(set(director_i).intersection(set(director_j))) / min_director_length
        else:
            director_ratio = 0.0
        
        #------------------------------------------------------------------------------
            
        total_ratio = 0.25 * keyword_ratio + 0.25 * genre_ratio + 0.2 * actors_ratio + 0.3 * director_ratio
        total_ratios.append([keyword_ratio, genre_ratio, actors_ratio, director_ratio])
        
        # if total ratio is more than the 'min_ratio' we insert a edge, between the nodes
        if total_ratio > min_ratio:
            similarity = 1 - spatial.distance.cosine(TF_IDF_matrix[movie_number[movies[i]]], TF_IDF_matrix[movie_number[movies[j]]])
            # the wieght is 1.4 - total_ratio, since if we get higher ratio we are closer in the network, and similarity, if the
            # storylines are similar the films are closer in the network, we chose 1.4 so we dont have any negative weights
            # since calculating the shortest path with dijekstra requiers positive weights
            G.add_edge(movies[i], movies[j], weight = 1.4 - total_ratio - 0.4 * similarity)
            links.append((total_ratio, [keyword_ratio, genre_ratio, actors_ratio, director_ratio] ))

            


print "Whata Network"
#nx.draw(G_keywords, node_size = 30, node_color = 'blue')
#plt.show()


Whata Network


### Looking at the results
Looking at the results we can see that most of them make very much sens. It is suprising to that iron man 2 recommends zathura, however zathura has the same director and genere among other things.

In [22]:
sor = []
for key in G['iron man 2'].keys():
    sor.append((G['iron man 2'][key]['weight'], key))

for movie in sorted(sor):
    print movie

(0.47457941494519651, 'iron man')
(0.81190102008266174, 'iron man 3')
(0.91612143975999416, 'captain america civil war')
(0.93152758296692273, 'zathura a space adventure')
(0.93829986540305765, 'avengers age of ultron')
(0.94466152218008048, 'the avengers')
(0.94469177223863288, 'captain america the winter soldier')
(0.95692343750867681, 'captain america the first avenger')
(0.95721449579239393, 'antman')
(0.96479419264758626, 'astro boy')
(0.98681373056301369, 'xmen')
(0.98845616972245298, 'xmen apocalypse')
(0.98999241006372007, 'the incredible hulk')
(0.99394647710657746, 'the jungle book')
(0.99597466868628659, 'x2')
(0.99908940648700773, 'made')
(1.0094881271916345, 'the specials')
(1.0227282717122419, 'fantastic four')
(1.0232975221062659, 'the wolverine')
(1.0249999999999999, 'all superheroes must die')
(1.0310280851264046, 'thor')
(1.0363866935735622, 'xmen the last stand')
(1.0380134790721358, 'guardians of the galaxy')
(1.0383984822056851, 'xmen first class')
(1.0396862457337

### Calculate all the shortest paths in the network

In [55]:
movies_in_network = []
for _movie in movies:
    shortest_paths = nx.single_source_dijkstra_path_length(G, _movie)
    movies_in_network.append(shortest_paths)
    
print "All shortest paths calculated"
         

All shortest paths calculated


In [42]:
movies_in_network

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


### Add the imdb rating

It was decided to use the imdb rating to shorten the distance in the network. This will yield a shorter distnace to movies which have higher imdb rating. This should improve our search engine, since this shortens the distnace to good movies. 

In [56]:
# get the imdb scores
with open("C:\Users\helga\Dropbox\Ingvar\Results\imdb-score-mod-mod.json") as data_file:    
    IMBD_ratings = json.load(data_file)


imdb_importance = 0.15
all_possible_recommendations = []
num_options = 10
need_more_recommendations = []

for m in movies_in_network:
    sorted_distances = sorted(m.items(), key=operator.itemgetter(1))
    possible_recommendations = []
    
    for movie_distance in sorted_distances[1:]:
        # take all the elements which can be in top ten after the imdb rating is added to the network
        if len(possible_recommendations) < num_options or  sorted_distances[num_options - 1][1] > movie_distance[1] - imdb_importance:
            # get the imdb rating
            _id = tmdb_5000_movies[movie_distance[0]]['id']
            
            normalized_raiting =  imdb_importance * float(IMBD_ratings[_id]) / 10
          
            
            # calculate the new reccomandation value and store it, put it to a nice form to easly sort and change to dict l8ter on
            movie_distance = [movie_distance[1] - normalized_raiting, movie_distance[0]]
            possible_recommendations.append(movie_distance)
        else:
            # the movies here after are not good enough recommendation for our network
            break
    if len(possible_recommendations) < num_options:
        need_more_recommendations.append(sorted_distances[0][0])
    all_possible_recommendations.append((sorted_distances[0][0], possible_recommendations))

print "Shortest x members have been calculated"

Shortest x members have been calculated


### Sorted recommendations

In [57]:
# change the format
all_possible_recommendations = dict(all_possible_recommendations)

for key in all_possible_recommendations:
    all_possible_recommendations[key] = sorted(all_possible_recommendations[key])[0:num_options]

### Those who dont have enough recommendation
Some movies(nodes) do not have any links. However, we still want to recommend movies when people like to find similar movies to them.  In this case we depend only on the imdb rating and the similarity between movies. Thus we will only recommend a movie which should be somewhat similar and has a good imdb rating. Note that the distance here is calculated as three minus the rating and the similarity. The reason for the 3 is that, if some of these movies have only connection to one movie (they are an island) then we want that to be our first chose. Therefore we use 3 so that the value of the edge will never go below 1.

In [58]:
for first_movie in need_more_recommendations:
    for second_movie in movies:
        second_id = tmdb_5000_movies[second_movie]['id']
        normalized_rating = 0.9 * float(IMBD_ratings[_id]) / 10
        similarity = 1 - spatial.distance.cosine(TF_IDF_matrix[movie_number[first_movie]], TF_IDF_matrix[movie_number[second_movie]])
        # here we put the "distance" to the nodes which are not in the network, we choose to subtract from
        # 3 so we dont get better disntaance than of the movies which arer in the network
        movie_distance = [3 - normalized_rating - 0.7 * similarity, second_movie]
        all_possible_recommendations[first_movie].append(movie_distance)
    all_possible_recommendations[first_movie] = sorted(all_possible_recommendations[first_movie])[:num_options]
        
    

print "all recommendations ready"

all recommendations ready


### Make the json data of our complete network

In [59]:
with open('search_results3.json', 'w') as fp:
    json.dump(all_possible_recommendations, fp)
print "json ready"

json ready


### Get json data for the movies in our network
this is done for the autocomplete function on the webside

In [60]:
# Get data line by line from tmdb-5000-movie-database and save the data as list
filepath = r"C:\Users\helga\Dropbox\Ingvar\FinalProject\Data\tmdb-5000-movie-dataset\tmdb_5000_credits.csv"
_movies_in_network = []
_movies_in_network_json_format = {}
with open(filepath) as csvfile:
    reader = csv.DictReader(csvfile, delimiter=",")
    for row in reader:
        if row['title'] not in movies_to_delete:
            _movies_in_network.append(row["title"])
            
_movies_in_network_json_format['movies'] = _movies_in_network

with open('all_movies.json', 'w') as fp:
    json.dump(_movies_in_network_json_format, fp)
print "json ready"

json ready
